Using the Luchtmeetnet API: https://api-docs.luchtmeetnet.nl/#intro

## Over data

### NO2
De hoogste concentraties stikstofdioxide (NO2) komen voor tijdens de ochtend- en avondspits. Deze stof komt vrij door het (weg)verkeer, energieproductie en industrie. Daarnaast ontstaat NO2 uit een reactie tussen stikstofmonoxide en ozon. Het weer en de verkeersdrukte hebben grote invloed op de concentratie. De wettelijke norm is een jaargemiddelde van 40 (μg/m3)
* Slecht: > 80 (y tot 100)
* Matig: 30 - 80
* Goed: < 30

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### NO
De concentratie stikstofmonoxide (NO) is rond de ochtend- en avondspits hoger. Deze stof komt vrij bij het verbranden van brandstof door auto's, cv-installaties, de industrie en elektriciteitscentrales. Eenmaal in de lucht vindt er een chemisch proces plaats. stikstofmonoxide wordt dan omgezet in stikstofdioxide. Voor stikstofmonoxide bestaan geen wettelijke normen.
* Goed: < 50
* Matig: > 50 (y tot 100) 

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### O3
Ozon wordt niet rechtstreeks uitgestoten, maar wordt gevormd uit stikstofoxiden, vluchtige organische stoffen en koolmonoxide. De concentratie ozon (O3) is vooral afhankelijk van het weer. In heel Europa wordt de bevolking gewaarschuwd bij ozonconcentraties boven 180 (μg/m3). Een concentratie van 240 (μg/m3) is de Europese alarmdrempel.
* Goed: 0 - 40
* Matig: > 40 (y tot 100)

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### PM10
De dagelijkse concentratie fijn stof (PM10) is afhankelijk van het weer. In de steden zijn de concentraties overdag gemiddeld iets hoger dan 's nachts, vooral door de verkeersbijdrage. PM10 is een verzamelnaam voor zwevende, inhaleerbare deeltjes met een maximale doorsnede van 0,01 milimeter. De wettelijke norm is een jaargemiddelde van 40 (μg/m3). Daarnaast mag het daggemiddelde jaarlijks maximaal 35 keer hoger zijn dan 50 (μg/m3). 
* Goed: < 30
* Matig: 30 - 70
* Slecht: > 70

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

### PM2.5
De dagelijkse concentratie fijn stof (PM2.5) is afhankelijk van het weer. In de steden zijn de concentraties overdag gemiddeld iets hoger dan ’s nachts, vooral door de verkeersbijdrage. PM2.5 is een verzamelnaam voor zwevende, inhaleerbare deeltjes met een maximale doorsnede van 0,0025 millimeter. De wettelijke norm is een jaargemiddelde van 25 (μg/m3). Doordat PM2.5 nog kleiner is dan PM10 kunnen deze deeltjes dieper doordringen in de longen en zijn ze schadelijker voor de gezondheid.

PM2.5 apparatuur is gevoelig voor condensvorming. Bij hoge buitentemperaturen is de kans op condensvorming groter. De meetapparatuur keurt een meting automatisch af wanneer condensvorming verwacht wordt. Volgens de automatische instellingen in de meetapparatuur wordt condensvorming verwacht wanneer het verschil tussen de buitenluchttemperatuur en temperatuur in het meetinstrument klein is. Bij hoge buitenluchttemperaturen is de kans groot dat het temperatuurverschil klein is en de metingen automatisch worden afgekeurd. Als gevolg hiervan ontbreken PM2.5 metingen soms als de buitentemperatuur hoog is. Indien later bij controle door de meetnetbeheerder blijkt dat een meting toch correct heeft plaatsgevonden worden metingen alsnog goedgekeurd.
* Goed: < 20
* Matig: 20 - 50
* Slecht: > 50

Unit: Concentraties in µg/m³, UFP in aantal/cm³.

In [11]:
import json
import requests
from dateutil.parser import parse
from collections import defaultdict
from datetime import datetime, timezone, timedelta

In [12]:
class API:
    url = f"https://api.luchtmeetnet.nl/open_api/measurements"
        
    def get(self, station=None, start=None, end=None, formula=None):
        request_url = self.url
        
        # blegh - query string params
        # TODO something with station
        qsp_to_add = ["station_number=NL10404"]
        
        if start and end:
            qsp_to_add.append(f"start={start}")
            qsp_to_add.append(f"end={end}")
        if formula:
            qsp_to_add.append(f"formula={formula}")
            
        if len(qsp_to_add) > 0:
            stuff = "&".join(qsp_to_add)
            request_url += f"?{stuff}" 
        
        try:
            response = requests.get(request_url) 
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print('Something went wrong while getting data from api: ', e)
            return []
        
        response_json = response.json()
            
        if 'pagination' in response_json:
            current_page = response_json['pagination']['current_page']
            last_page = response_json['pagination']['last_page']
            if current_page != last_page:
                print("TODO - pagination needed")
        
        response_data = response_json['data']
                
        return response_data

In [13]:
class LuchtmeetnetData:
    
    metrics = dict()
    measurements = defaultdict(dict)
    
    def __init__(self, api, formulas):
        self.api = api
        for formula in formulas:
            self.metrics[formula] = Metric(formula)
        self.add(api.get())
            
    def add(self, data):
        for item in data:
            formula = item['formula']
            value = item['value']
            timestamp = item['timestamp_measured']

            metric = self.metrics[formula]
            
            self.measurements[formula][timestamp] = Measurement(metric, value, timestamp)
    
    def get_timestamp(self, metric, timestamp):
        if timestamp in self.measurements[metric]:
#             print("already there")
            delta_measurement = self.measurements[metric][timestamp]
            return delta_measurement
        else:
#             print("api call")
            new_data = self.api.get(start=timestamp, end=timestamp)

            if len(new_data) > 0:
                self.add(new_data)
                return self.get_timestamp(metric, timestamp)
            return None
        
    def get_current_report(self, time_deltas):
        the_report = defaultdict()
        timestamp = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0).isoformat()
        the_report['timestamp'] = timestamp
        the_report['measurements'] = defaultdict(dict)
        
        for metric in self.measurements:
            if timestamp in self.measurements[metric]:
                current_measurement = self.measurements[metric][timestamp]
                the_report['measurements'][metric]["NOW"] = {
                    'value': current_measurement.value,
                    'level': current_measurement.evaluate()
                }

                for enum, time_delta in time_deltas:
                    result_time = current_measurement.timestamp - time_delta
                    delta_time_string = result_time.isoformat()

                    delta_measurement = self.get_timestamp(metric, delta_time_string)
                    the_report['measurements'][metric][enum] = {
                        'value': current_measurement.compare(delta_measurement),
                        'level': current_measurement.evaluate_change(delta_measurement)
                    }
        
        return the_report
                

                    

In [14]:
class Metric:
    metric_types = dict()
    metric_types['NO2'] = {'ub_good': 30, 'ub_med': 80, 'ub_bad': 100}
    metric_types['NO'] = {'ub_good': 30, 'ub_med': 100}
    metric_types['O3'] = {'ub_good': 40, 'ub_med': 180}
    metric_types['PM10'] = {'ub_good': 30, 'ub_med': 70, 'ub_bad': 100}
    metric_types['PM25'] = {'ub_good': 20, 'ub_med': 50, 'ub_bad': 100}

    ub_bad = float('inf') # why not?
    
    def __repr__(self):
        return f"{self.formula}"
    
    def __init__(self, formula):    
        self.formula = formula
        metric_info = self.metric_types[self.formula]
        if 'ub_good' in metric_info:
            self.ub_good = metric_info['ub_good']
        if 'ub_med' in metric_info:
            self.ub_med = metric_info['ub_med']
        if 'ub_bad' in metric_info:
            self.ub_bad = metric_info['ub_bad']
            
    def judgement(self, value):
        if value <= self.ub_good:
            return "GOOD"
        elif value <= self.ub_med:
            return "MEDIOCRE"
        elif value <= self.ub_bad:
            return "BAD"
        else:
            return f"PANIC - {value}"
    

In [15]:
class Measurement:

    def __repr__(self):
        return f"{self.metric.formula}, {self.value}, {self.timestamp}"
    
    def __init__(self, metric, value, timestamp):
        self.metric = metric
        self.value = float(value) # better safe than sorry
        self.timestamp = parse(timestamp)
        
    def evaluate(self):
        judgement = self.metric.judgement(self.value)
        return judgement
    
    def compare(self, other_measurement):
        diff = self.value - other_measurement.value
        return diff
    
    def evaluate_change(self, other_measurement):
        diff = self.compare(other_measurement)
        change = diff / self.value
        if change <= -0.5:
            return "ANOMALOUS_DECREASE"
        elif change >= 0.5:
            return "ANOMALOUS_INCREASE"
        else:
            return "NO_ANOMALY"
        


In [16]:
# CONFIG
formulas = ['PM10', 'PM25', 'NO2', 'NO', 'O3']

time_deltas = []
time_deltas.append(("ONE_HOUR", timedelta(hours=1))) # one hour
time_deltas.append(("ONE_DAY", timedelta(days=1))) # one day
time_deltas.append(("ONE_WEEK", timedelta(days=7))) # one day
time_deltas.append(("ONE_MONTH", timedelta(days=30))) # one day
time_deltas.append(("ONE_YEAR", timedelta(days=365))) # one day

api = API() 
data = LuchtmeetnetData(api, formulas)

In [17]:
current_report = data.get_current_report(time_deltas)

In [18]:
print(current_report)

defaultdict(None, {'timestamp': '2020-11-28T15:00:00+00:00', 'measurements': defaultdict(<class 'dict'>, {'NO': {'NOW': {'value': 18.19, 'level': 'GOOD'}, 'ONE_HOUR': {'value': -3.8599999999999994, 'level': 'NO_ANOMALY'}, 'ONE_DAY': {'value': 13.200000000000001, 'level': 'ANOMALOUS_INCREASE'}, 'ONE_WEEK': {'value': 17.200000000000003, 'level': 'ANOMALOUS_INCREASE'}, 'ONE_MONTH': {'value': 16.830000000000002, 'level': 'ANOMALOUS_INCREASE'}, 'ONE_YEAR': {'value': 15.650000000000002, 'level': 'ANOMALOUS_INCREASE'}}, 'PM10': {'NOW': {'value': 26.19, 'level': 'GOOD'}, 'ONE_HOUR': {'value': 6.400000000000002, 'level': 'NO_ANOMALY'}, 'ONE_DAY': {'value': -2.5599999999999987, 'level': 'NO_ANOMALY'}, 'ONE_WEEK': {'value': 6.400000000000002, 'level': 'NO_ANOMALY'}, 'ONE_MONTH': {'value': 5.120000000000001, 'level': 'NO_ANOMALY'}, 'ONE_YEAR': {'value': 11.520000000000001, 'level': 'NO_ANOMALY'}}, 'PM25': {'NOW': {'value': 22.25, 'level': 'MEDIOCRE'}, 'ONE_HOUR': {'value': -0.7699999999999996, 'le

In [19]:
print(json.dumps(current_report, indent=2))

{
  "timestamp": "2020-11-28T15:00:00+00:00",
  "measurements": {
    "NO": {
      "NOW": {
        "value": 18.19,
        "level": "GOOD"
      },
      "ONE_HOUR": {
        "value": -3.8599999999999994,
        "level": "NO_ANOMALY"
      },
      "ONE_DAY": {
        "value": 13.200000000000001,
        "level": "ANOMALOUS_INCREASE"
      },
      "ONE_WEEK": {
        "value": 17.200000000000003,
        "level": "ANOMALOUS_INCREASE"
      },
      "ONE_MONTH": {
        "value": 16.830000000000002,
        "level": "ANOMALOUS_INCREASE"
      },
      "ONE_YEAR": {
        "value": 15.650000000000002,
        "level": "ANOMALOUS_INCREASE"
      }
    },
    "PM10": {
      "NOW": {
        "value": 26.19,
        "level": "GOOD"
      },
      "ONE_HOUR": {
        "value": 6.400000000000002,
        "level": "NO_ANOMALY"
      },
      "ONE_DAY": {
        "value": -2.5599999999999987,
        "level": "NO_ANOMALY"
      },
      "ONE_WEEK": {
        "value": 6.400000000000002

In [20]:
for formula in current_report['measurements']:
    print(current_report['measurements'][formula])
    evaluated_measurement = current_report['measurements'][formula]['ONE_DAY']
    if not evaluated_measurement['level'] is "NO_ANOMALY":
        print("Send notification")

{'NOW': {'value': 18.19, 'level': 'GOOD'}, 'ONE_HOUR': {'value': -3.8599999999999994, 'level': 'NO_ANOMALY'}, 'ONE_DAY': {'value': 13.200000000000001, 'level': 'ANOMALOUS_INCREASE'}, 'ONE_WEEK': {'value': 17.200000000000003, 'level': 'ANOMALOUS_INCREASE'}, 'ONE_MONTH': {'value': 16.830000000000002, 'level': 'ANOMALOUS_INCREASE'}, 'ONE_YEAR': {'value': 15.650000000000002, 'level': 'ANOMALOUS_INCREASE'}}
Send notification
{'NOW': {'value': 26.19, 'level': 'GOOD'}, 'ONE_HOUR': {'value': 6.400000000000002, 'level': 'NO_ANOMALY'}, 'ONE_DAY': {'value': -2.5599999999999987, 'level': 'NO_ANOMALY'}, 'ONE_WEEK': {'value': 6.400000000000002, 'level': 'NO_ANOMALY'}, 'ONE_MONTH': {'value': 5.120000000000001, 'level': 'NO_ANOMALY'}, 'ONE_YEAR': {'value': 11.520000000000001, 'level': 'NO_ANOMALY'}}
a
{'NOW': {'value': 22.25, 'level': 'MEDIOCRE'}, 'ONE_HOUR': {'value': -0.7699999999999996, 'level': 'NO_ANOMALY'}, 'ONE_DAY': {'value': 9.56, 'level': 'NO_ANOMALY'}, 'ONE_WEEK': {'value': 16.15, 'level': 